In [10]:
import copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.io import read_image
from torchsummary import summary

from tqdm import tqdm
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import wandb
from ece9603_project import lossFunctions

device = "cpu"
if torch.cuda.is_available():
    device="cuda"

In [11]:
# Run once to setup connection to wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [12]:
"""
# Setup sweep hyperparameters
sweep_config = {
    'name': 'ensemble_sweep',
    'method': 'grid',
    'parameters': {
        'learning_rate': {
            'values': [0.001]
        },
        'epochs': {
            'values': [10]
        },
        'loss_function': {
            'values': ['dice_loss',
                       'bce_dice_loss',
                       'jaccard_iou_loss',
                       'focal_loss',
                       'tversky_loss',
                       'focal_tversky_loss',
                       'bce_with_logits_loss']
        }
    }
}
sweep_id = wandb.sweep(sweep_config,
                       project="breast-histopathology-classification",
                       entity="ece9603_project")
"""

'\n# Setup sweep hyperparameters\nsweep_config = {\n    \'name\': \'ensemble_sweep\',\n    \'method\': \'grid\',\n    \'parameters\': {\n        \'learning_rate\': {\n            \'values\': [0.001]\n        },\n        \'epochs\': {\n            \'values\': [10]\n        },\n        \'loss_function\': {\n            \'values\': [\'dice_loss\',\n                       \'bce_dice_loss\',\n                       \'jaccard_iou_loss\',\n                       \'focal_loss\',\n                       \'tversky_loss\',\n                       \'focal_tversky_loss\',\n                       \'bce_with_logits_loss\']\n        }\n    }\n}\nsweep_id = wandb.sweep(sweep_config,\n                       project="breast-histopathology-classification",\n                       entity="ece9603_project")\n'

In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.info = df
        self.transform = transform

    def __len__(self):
        return len(self.info)

    def __getitem__(self, idx):
        path = self.info.imgPath.values[idx]
        label = self.info['class'].values[idx]
        image = read_image(path, mode=torchvision.io.image.ImageReadMode.RGB).float()

        if self.transform:
            image = self.transform(image)

        return image, label

In [14]:
df= pd.read_csv("breastCancerDataframe.csv", index_col=0)
print(df.head())

patientIDs = df.patient.unique()
print("Number of Unique Patients: ", len(patientIDs))

patients_train, temp = train_test_split(patientIDs, test_size=0.3, random_state=42)
patients_val, patients_test = train_test_split(temp, test_size=0.5, random_state=42)

df_train = df.loc[df['patient'].isin(patients_train)]
print(df_train.head())
print("Number of Train Patients: ", df_train.patient.nunique())

df_val = df.loc[df['patient'].isin(patients_val)]
print(df_val.head())
print("Number of Validation Patients: ", df_val.patient.nunique())

df_test = df.loc[df['patient'].isin(patients_test)]
print(df_test.head())
print("Number of Test Patients: ", df_test.patient.nunique())

   patient  class  posX  posY                                         imgPath
0    12954      0  1151  1401  data/12954/0/12954_idx5_x1151_y1401_class0.png
1    12954      0  1951  2901  data/12954/0/12954_idx5_x1951_y2901_class0.png
2    12954      0   151   501    data/12954/0/12954_idx5_x151_y501_class0.png
3    12954      0  1701  2251  data/12954/0/12954_idx5_x1701_y2251_class0.png
4    12954      0  1501  2001  data/12954/0/12954_idx5_x1501_y2001_class0.png
Number of Unique Patients:  279
   patient  class  posX  posY                                         imgPath
0    12954      0  1151  1401  data/12954/0/12954_idx5_x1151_y1401_class0.png
1    12954      0  1951  2901  data/12954/0/12954_idx5_x1951_y2901_class0.png
2    12954      0   151   501    data/12954/0/12954_idx5_x151_y501_class0.png
3    12954      0  1701  2251  data/12954/0/12954_idx5_x1701_y2251_class0.png
4    12954      0  1501  2001  data/12954/0/12954_idx5_x1501_y2001_class0.png
Number of Train Patients:  195
 

In [15]:
BATCH_SIZE = 128

transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = CustomImageDataset(df_train, transform=transform)
val_dataset = CustomImageDataset(df_val, transform=transform)
test_dataset = CustomImageDataset(df_test, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [16]:
"""
print("Percent Neg: ", (df_train['class'].to_list()).count(0)/len(df_train))
print("Percent Pos: ", (df_train['class'].to_list()).count(1)/len(df_train))

df_train_neg = df_train.loc[df_train['class']==0]
df_train_pos = df_train.loc[df_train['class']==1]
print("Neg Count: ", len(df_train_neg), "Pos Count: ",len(df_train_pos))

#split it into 3 since i would rather the model have an over represented positive class so the model
#predicts pos more than neg since identifying pos is more important
df_train1, temp = train_test_split(df_train_neg, train_size=1/3, random_state=42)
df_train2, df_train3 = train_test_split(temp, train_size=1/2, random_state=42)
df_train1 = pd.concat([df_train1, df_train_pos], ignore_index=True)
df_train2 = pd.concat([df_train2, df_train_pos], ignore_index=True)
df_train3 = pd.concat([df_train3, df_train_pos], ignore_index=True)


fig, ax = plt.subplots(1,4,figsize=(25,5))
sns.countplot(df_train['class'], ax=ax[0], palette="Reds")
ax[0].set_title("Original Train Data")
sns.countplot(df_train1['class'], ax=ax[1], palette="Blues")
ax[1].set_title("Balanced Train Data 1")
sns.countplot(df_train2['class'], ax=ax[2], palette="Greens")
ax[2].set_title("Balanced Train Data 2")
sns.countplot(df_train3['class'], ax=ax[3], palette="Oranges")
ax[3].set_title("Balanced Train Data 3")

print("\nClass Percentages After Splitting")
print("Percent Neg Train1: ", (df_train1['class'].to_list()).count(0)/len(df_train1), "Percent Pos Train 1: ", (df_train1['class'].to_list()).count(1)/len(df_train1))
print("Percent Neg Train2: ", (df_train2['class'].to_list()).count(0)/len(df_train2), "Percent Pos Train 2: ", (df_train2['class'].to_list()).count(1)/len(df_train2))
print("Percent Neg Train3: ", (df_train3['class'].to_list()).count(0)/len(df_train3), "Percent Pos Train 3: ", (df_train3['class'].to_list()).count(1)/len(df_train3))

train_dataloaders = [DataLoader(CustomImageDataset(df_train1, transform=transform), batch_size=BATCH_SIZE, shuffle=True),
                     DataLoader(CustomImageDataset(df_train2, transform=transform), batch_size=BATCH_SIZE, shuffle=True),
                     DataLoader(CustomImageDataset(df_train3, transform=transform), batch_size=BATCH_SIZE, shuffle=True)]
"""

'\nprint("Percent Neg: ", (df_train[\'class\'].to_list()).count(0)/len(df_train))\nprint("Percent Pos: ", (df_train[\'class\'].to_list()).count(1)/len(df_train))\n\ndf_train_neg = df_train.loc[df_train[\'class\']==0]\ndf_train_pos = df_train.loc[df_train[\'class\']==1]\nprint("Neg Count: ", len(df_train_neg), "Pos Count: ",len(df_train_pos))\n\n#split it into 3 since i would rather the model have an over represented positive class so the model\n#predicts pos more than neg since identifying pos is more important\ndf_train1, temp = train_test_split(df_train_neg, train_size=1/3, random_state=42)\ndf_train2, df_train3 = train_test_split(temp, train_size=1/2, random_state=42)\ndf_train1 = pd.concat([df_train1, df_train_pos], ignore_index=True)\ndf_train2 = pd.concat([df_train2, df_train_pos], ignore_index=True)\ndf_train3 = pd.concat([df_train3, df_train_pos], ignore_index=True)\n\n\nfig, ax = plt.subplots(1,4,figsize=(25,5))\nsns.countplot(df_train[\'class\'], ax=ax[0], palette="Reds")\nax

In [17]:
#split training data for ensomble models
def ensemble_train_split(df_train):
    #seperate positive and negative samples
    df_train_neg = df_train.loc[df_train['class']==0]
    df_train_pos = df_train.loc[df_train['class']==1]

    #make positive class 10% the size of neg to make skew in pos class more noticeable
    df_train_pos_10Percent = df_train_pos.sample(n=int(len(df_train_neg)*0.1), replace=False, random_state=42)
    num_folds=int(len(df_train_neg)/len(df_train_pos_10Percent))
    print(num_folds)

    samples_per_split = int(len(df_train_neg)/num_folds)
    #save train dfs incase we want to graph later
    train_dataloaders, train_dfs = [], []

    for i in range(num_folds):
        neg_fold = df_train_neg[i*samples_per_split:(i+1)*samples_per_split]
        train_dfs.append(pd.concat([neg_fold, df_train_pos_10Percent], ignore_index=True))
        train_dataloaders.append(DataLoader(CustomImageDataset(train_dfs[i], transform=transform), batch_size=BATCH_SIZE, shuffle=True))
        print("Percent Neg Train fold_"+str(i)+": ", ((train_dfs[i])['class'].to_list()).count(0)/len(train_dfs[i]), "Percent Pos Train fold_"+str(i)+": ", ((train_dfs[i])['class'].to_list()).count(1)/len(train_dfs[i]))

    return train_dataloaders

# Calculate performance measures
def compute_performance(yhat, y, pos_cutoff, evaluation_phase='validation',
                        model_id=0):

    # First, get tp, tn, fp, fn
    tp = sum(np.logical_and(yhat >= pos_cutoff, y == 1).numpy())
    tn = sum(np.logical_and(yhat < pos_cutoff, y == 0).numpy())
    fp = sum(np.logical_and(yhat >= pos_cutoff, y == 0).numpy())
    fn = sum(np.logical_and(yhat < pos_cutoff, y == 1).numpy())

    print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")

    # Precision
    # "Of the ones I labeled +, how many are actually +?"
    precision = tp / (tp + fp)

    # Recall
    # "Of all the + in the data, how many do I correctly label?"
    recall = tp / (tp + fn)

    # Sensitivity
    # "Of all the + in the data, how many do I correctly label?"
    sensitivity = recall

    # Specificity
    # "Of all the - in the data, how many do I correctly label?"
    specificity = tn / (fp + tn)

    balanced_acc = 0.5*(sensitivity+specificity)
    #fMeasure =  2*((precision*recall)/(precision+recall))

    auroc = roc_auc_score(y, yhat)

    # Print results
    print("Balanced Accuracy: ", balanced_acc," Specificity: ",specificity, " AUROC Score: ", auroc,
          " Sensitivity: ", sensitivity," Precision: ", precision)
    # Log results to WandB
    wandb.log({"(model-{}-{}) Balanced Accuracy".format(model_id, evaluation_phase): balanced_acc,
               "(model-{}-{}) Specificity".format(model_id, evaluation_phase): specificity,
               "(model-{}-{}) Sensitivity".format(model_id, evaluation_phase): sensitivity,
               "(model-{}-{}) Precision".format(model_id, evaluation_phase): precision,
               "(model-{}-{}) AUROC Score".format(model_id, evaluation_phase): auroc},
              commit=False)

def train(model, dataloader_train, dataloader_val, device='cpu', epochs=10, early_stop=2, lr=0.001,
          loss_function='bce_with_logits_loss', model_id=0, verbose=True):

    opt = torch.optim.Adam(model.classifier.parameters(), lr=lr)

    criterion = lossFunctions.getLossFunction(loss_function)

    model.to(device)

    lowest_val_loss, train_loss = np.inf, 0
    lowest_val_epoch = 0
    epochs_wo_improvement = 0
    best_model = copy.deepcopy(model.state_dict())
    train_losses, val_losses=[], []
    train_preds, train_targets_list = [], []

    for e in range(epochs):
        epoch_train_loss = 0
        epoch_val_loss = 0

        model.train()

        for inputs, targets in tqdm(dataloader_train):

            inputs, targets = inputs.to(device), targets.to(device)

            model.zero_grad(set_to_none=True)

            train_output = model.forward(inputs).squeeze()
            train_preds+=train_output
            train_targets_list+=targets
            loss = criterion(train_output, targets.float())
            loss.backward()
            opt.step()

            epoch_train_loss+=loss

        compute_performance(torch.sigmoid(torch.Tensor(train_preds)), torch.Tensor(train_targets_list), 0.5, evaluation_phase='training', model_id=model_id)
        epoch_train_loss = epoch_train_loss.item()/((len(dataloader_train.dataset)%BATCH_SIZE)*BATCH_SIZE)
        train_losses.append(epoch_train_loss)

        #VALIDATION

        model.eval()
        model.zero_grad(set_to_none=True)
        val_preds, val_targets_list = [], []

        with torch.no_grad():
            for val_inputs, val_targets in tqdm(dataloader_val):

                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)

                val_output = model.forward(val_inputs).squeeze()
                val_preds+=val_output
                val_targets_list+=val_targets

                epoch_val_loss += criterion(val_output, val_targets.float())

            epoch_val_loss = epoch_val_loss.item()/((len(dataloader_val.dataset)%BATCH_SIZE)*BATCH_SIZE)
            val_losses.append(epoch_val_loss)

            compute_performance(torch.sigmoid(torch.Tensor(val_preds)), torch.Tensor(val_targets_list), 0.5,
                                evaluation_phase='validation', model_id=model_id)

        if epoch_val_loss <= lowest_val_loss:
            best_model = copy.deepcopy(model.state_dict())
            lowest_val_loss = epoch_val_loss
            train_loss=epoch_train_loss
            lowest_val_epoch=e
            epochs_wo_improvement=0
        else:
            epochs_wo_improvement+=1

        if verbose:
            print("Epoch: {}/{}...".format(e, epochs), "Loss: {:.4f}...".format(epoch_train_loss), "Val Loss: {:.4f}".format(epoch_val_loss),)

        # Log to wandb project
        wandb.log({"(model-{}) training_loss".format(model_id): epoch_train_loss,
                   "(model-{}) validation_loss".format(model_id): epoch_val_loss})

        #early stopping
        if epochs_wo_improvement>=early_stop:
            if verbose:
                print("Early Stop no improvement in validation loss in "+str(early_stop)+" validation steps")
            break

    if verbose:
        print("\nLowest Validation Loss: "+str(lowest_val_loss)+" at epoch "+str(lowest_val_epoch)+'\n')

    model.load_state_dict(best_model)
    # Record model to wandb
    wandb.watch(model)

    run_ID = datetime.now().strftime("%Y-%m-%d_%H-%M")
    torch.save({'model_state_dict': best_model}, './BestModels/'+str(run_ID)+'_E_'+str(lowest_val_epoch)+'_TL_'+str(round(train_loss, 4))+'_VL_'+str(round(lowest_val_loss, 4))+'.pt')

    model.to("cpu")
    torch.cuda.empty_cache()

    return model, train_losses, val_losses

def init_models(dataloaders, load_save=False):

    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    if load_save:
        ensemble = [models.efficientnet_b0(pretrained=True) for _ in dataloaders]

        for model in ensemble:
            for param in model.parameters():
                param.requires_grad = False

            model.classifier = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(1280, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),

                nn.Dropout(0.2),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),

                nn.Linear(256, 1))

            model.apply(init_weights)
    else:
        ensemble = [[models.efficientnet_b0(pretrained=True), data] for data in dataloaders]

        for model, _ in ensemble:
            for param in model.parameters():
                param.requires_grad = False

            model.classifier = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(1280, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),

                nn.Dropout(0.2),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),

                nn.Linear(256, 1))

            model.apply(init_weights)
    return ensemble

def get_trained_ensemble(dataloaders_train, dataloader_val, loss_function='bce_with_logits_loss'):
    ensemble = init_models(dataloaders=dataloaders_train)
    trained_ensemble = []

    torch.backends.cudnn.benchmark = True
    model_id = 1
    for mod, dataloader_train in ensemble:
        model, train_losses, val_losses = train(mod, dataloader_train, dataloader_val, early_stop=1,
                                                device=device, loss_function=loss_function, model_id=model_id)
        trained_ensemble.append(model)
        model_id += 1
    return trained_ensemble

def ensemble_predict(models, dataloader_test, device='cpu', loss_function='bce_with_logits_loss'):
    for mod in models:
        mod.to(device)
        mod.eval()
        mod.zero_grad(set_to_none=True)

    test_loss = 0
    test_preds, test_targets_list = [], []
    criterion = lossFunctions.getLossFunction(loss_function)

    with torch.no_grad():
        for test_inputs, test_targets in tqdm(dataloader_test):

            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            batch_output=[]

            for model in models:
                batch_output.append(torch.sigmoid(model.forward(test_inputs).squeeze()).cpu().numpy())

            #average models output
            batch_output = np.column_stack(batch_output)
            test_output = np.mean(batch_output, axis=1)

            test_preds = np.hstack((test_preds, test_output))
            test_targets_list+=test_targets
            test_loss += criterion(torch.tensor(test_output).to(device), test_targets.float())

        print("Test Loss: ", test_loss.item()/((len(dataloader_test.dataset)%BATCH_SIZE)*BATCH_SIZE))

        compute_performance(torch.Tensor(test_preds), torch.Tensor(test_targets_list), 0.5, 
                            evaluation_phase="testing")


def trainAndTestModel():
    config_defaults = {
        "learning_rate": 0.001,
        "epochs": 10,
        "loss_function": "bce_with_logits_loss"
    }
    wandb.init(project="breast-histopathology-classification",
               entity="ece9603_project",
               job_type="model_training",
               config=config_defaults)
    config = wandb.config
    
    trained_ensemble = get_trained_ensemble(ensemble_train_split(df_train), val_dataloader, loss_function=config.loss_function)
    ensemble_predict(trained_ensemble, test_dataloader, device='cuda', loss_function=config.loss_function)

    # Done this training run
    wandb.finish()

## Don't really need this

``` python
load_save = False

if load_save:
    checkpoint1 = torch.load('BestModels/Ensemble_1/2021-11-21_17-33_E_3_TL_0.0162_VL_0.01.pt')
    checkpoint2 = torch.load('BestModels/Ensemble_1/2021-11-21_17-41_E_0_TL_0.0187_VL_0.01.pt')
    checkpoint3 = torch.load('BestModels/Ensemble_1/2021-11-21_18-03_E_3_TL_0.0161_VL_0.0097.pt')

    trained_ensemble = init_models(dataloaders=train_dataloaders, load_save=load_save)

    trained_ensemble[0].load_state_dict(checkpoint1['model_state_dict'])
    trained_ensemble[1].load_state_dict(checkpoint2['model_state_dict'])
    trained_ensemble[2].load_state_dict(checkpoint3['model_state_dict'])
else:
    trained_ensemble = get_trained_ensemble(val_dataloader)

ensemble_predict(trained_ensemble, test_dataloader, device='cuda')
```

In [18]:
# Run the sweep
#wandb.agent(sweep_id, trainAndTestModel)

trainAndTestModel()

(model-1) training_loss,▁
(model-1) validation_loss,▁
(model-1-training) AUROC Score,▁█
(model-1-training) Balanced Accuracy,▁█
(model-1-training) Precision,▁█
(model-1-training) Sensitivity,▁█
(model-1-training) Specificity,▁█
(model-1-validation) AUROC Score,▁
(model-1-validation) Balanced Accuracy,▁
(model-1-validation) Precision,▁
(model-1-validation) Sensitivity,▁


10
Percent Neg Train fold_0:  0.5 Percent Pos Train fold_0:  0.5
Percent Neg Train fold_1:  0.5 Percent Pos Train fold_1:  0.5
Percent Neg Train fold_2:  0.5 Percent Pos Train fold_2:  0.5
Percent Neg Train fold_3:  0.5 Percent Pos Train fold_3:  0.5
Percent Neg Train fold_4:  0.5 Percent Pos Train fold_4:  0.5
Percent Neg Train fold_5:  0.5 Percent Pos Train fold_5:  0.5
Percent Neg Train fold_6:  0.5 Percent Pos Train fold_6:  0.5
Percent Neg Train fold_7:  0.5 Percent Pos Train fold_7:  0.5
Percent Neg Train fold_8:  0.5 Percent Pos Train fold_8:  0.5
Percent Neg Train fold_9:  0.5 Percent Pos Train fold_9:  0.5


  0%|          | 0/334 [00:00<?, ?it/s]

tp: 12098 tn: 12196 fp: 2072 fn: 2170
Balanced Accuracy:  0.8513456686291001  Specificity:  0.8547799271096159  AUROC Score:  0.9267025587478404  Sensitivity:  0.8479114101485843  Precision:  0.8537755822159492


  0%|          | 0/223 [00:00<?, ?it/s]

...

100%|██████████| 334/334 [01:13<00:00,  4.54it/s]


tp: 12268 tn: 24069 fp: 4505 fn: 1858
Balanced Accuracy:  0.855404339179345  Specificity:  0.8423391894729474  AUROC Score:  0.9317286617643461  Sensitivity:  0.8684694888857426  Precision:  0.7314135813509808
Epoch: 2/10... Loss: 0.0043... Val Loss: 0.0120
Early Stop no improvement in validation loss in 1 validation steps

Lowest Validation Loss: 0.011310000168649774 at epoch 1



100%|██████████| 301/301 [03:04<00:00,  1.63it/s]


Test Loss:  0.026626481163886287
tp: 10019 tn: 22317 fp: 5119 fn: 1073
Balanced Accuracy:  0.8583419685387077  Specificity:  0.8134203236623414  AUROC Score:  0.9363366756384477  Sensitivity:  0.9032636134150739  Precision:  0.661844365173735


(model-0-Ensemble Test) AUROC Score,▁
(model-0-Ensemble Test) Balanced Accuracy,▁
(model-0-Ensemble Test) Precision,▁
(model-0-Ensemble Test) Sensitivity,▁
(model-0-Ensemble Test) Specificity,▁
(model-1) training_loss,█▂▁
(model-1) validation_loss,▇▁█
(model-1-training) AUROC Score,▁▆█
(model-1-training) Balanced Accuracy,▁▆█
(model-1-training) Precision,▁▆█
(model-1-training) Sensitivity,▁▆█
